In [ ]:
# ============================
# Cell 1: Imports & Config
# ============================
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import efficientnet

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import json

print("TF Version:", tf.__version__)

# --------- CONFIG ---------
DATA_ROOT = "/kaggle/input/cattle-classification/final_split"
TRAIN_DIR = os.path.join(DATA_ROOT, "train")
VAL_DIR   = os.path.join(DATA_ROOT, "val")

IMG_SIZE = (300, 300)  # EfficientNetB3 default
BATCH_SIZE = 16
EPOCHS = 30
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE


In [ ]:
# ============================
# Cell 3: Dataset Pipeline
# ============================

def create_datasets(train_dir, val_dir, img_size, batch_size, seed):
    train_ds = keras.utils.image_dataset_from_directory(
        train_dir,
        labels="inferred",
        label_mode="int",
        image_size=img_size,
        batch_size=batch_size,
        shuffle=True,
        seed=seed,
    )

    val_ds = keras.utils.image_dataset_from_directory(
        val_dir,
        labels="inferred",
        label_mode="int",
        image_size=img_size,
        batch_size=batch_size,
        shuffle=False,
        seed=seed,
    )

    class_names = train_ds.class_names
    num_classes = len(class_names)
    print("Classes:", class_names)
    print("Total breeds:", num_classes)

    train_ds = train_ds.prefetch(AUTOTUNE)
    val_ds   = val_ds.prefetch(AUTOTUNE)

    return train_ds, val_ds, class_names, num_classes

train_ds, val_ds, class_names, NUM_CLASSES = create_datasets(
    TRAIN_DIR, VAL_DIR, IMG_SIZE, BATCH_SIZE, SEED
)


In [ ]:
# ============================
# Cell 4: Data Augmentation
# ============================

from keras_cv.layers import RandomBrightness

def get_data_augmentation():
    return keras.Sequential(
        [
            layers.RandomFlip("horizontal"),
            layers.RandomRotation(0.08),
            layers.RandomZoom(0.1),
            layers.RandomTranslation(0.08, 0.08),
            layers.RandomCrop(300, 300),

            # built-in brightness from KerasCV
            RandomBrightness(factor=0.12),
        ],
        name="data_augmentation"
    )

data_augmentation = get_data_augmentation()


In [ ]:
# ============================
# Cell 5: Build EfficientNetB3 Model
# ============================

def build_efficientnet_b3(img_size, num_classes):
    inputs = layers.Input(shape=img_size + (3,), name="input_image")

    x = data_augmentation(inputs)
    x = efficientnet.preprocess_input(x)

    base = efficientnet.EfficientNetB3(
        include_top=False,
        weights="imagenet",
        input_tensor=x,
        pooling="avg"
    )
    base.trainable = False

    x = layers.Dropout(0.3)(base.output)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer=keras.optimizers.Adam(1e-3),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

model = build_efficientnet_b3(IMG_SIZE, NUM_CLASSES)
model.summary()


In [ ]:
# ============================
# Cell 6: Callbacks
# ============================

CHECKPOINT_PATH = "best_b3_cattle.keras"   # use .keras

callbacks = [
    keras.callbacks.ModelCheckpoint(
        CHECKPOINT_PATH,
        monitor="val_accuracy",
        save_best_only=True,
        verbose=1,
    ),
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=5,
        restore_best_weights=True,
        verbose=1,
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.3,
        patience=2,
        verbose=1,
        min_lr=1e-6
    ),
]


In [ ]:
# ============================
# Cell 7: Train
# ============================

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks,
)


In [ ]:
# ============================
# Cell 8: Load Best Weights
# ============================

model = keras.models.load_model(CHECKPOINT_PATH)


In [ ]:
# ============================
# Cell 9: Plot Training Curves
# ============================

def plot_training_curves(history):
    hist = history.history
    epochs = range(1, len(hist["loss"]) + 1)

    plt.figure(figsize=(14,5))
    plt.subplot(1,2,1)
    plt.plot(epochs, hist["loss"], label="Train")
    plt.plot(epochs, hist["val_loss"], label="Val")
    plt.title("Loss")
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(epochs, hist["accuracy"], label="Train")
    plt.plot(epochs, hist["val_accuracy"], label="Val")
    plt.title("Accuracy")
    plt.legend()

    plt.show()

plot_training_curves(history)


In [ ]:
# ============================
# Cell 10: Evaluation
# ============================

def get_predictions(model, dataset):
    y_true = []
    y_pred = []
    for imgs, labels in dataset:
        preds = model.predict(imgs, verbose=0)
        y_pred.extend(np.argmax(preds, axis=1))
        y_true.extend(labels.numpy().tolist())
    return np.array(y_true), np.array(y_pred)

y_true, y_pred = get_predictions(model, val_ds)

cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix Shape:", cm.shape)

# --- Heatmap with numbers ---
plt.figure(figsize=(16,14))
sns.heatmap(
    cm,
    annot=True,         # <----- numbers inside
    fmt="d",            # <----- integer format
    cmap="Blues",
    xticklabels=class_names,
    yticklabels=class_names,
)

plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix - Validation Set")
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# --- Also print raw numeric matrix (optional) ---
print("\nRaw Confusion Matrix Numbers:")
print(cm)

# --- Classification Report ---
print("\nClassification Report:")
print(
    classification_report(
        y_true,
        y_pred,
        target_names=class_names,
        digits=4,
    )
)

# --- Per-class accuracy ---
print("\nPer-class Accuracy:")
per_class_acc = (np.diag(cm) / cm.sum(axis=1)) * 100.0
for idx, breed in enumerate(class_names):
    print(f"{breed:25s}: {per_class_acc[idx]:.2f} %")


In [ ]:
# ============================
# Cell 11: Export FP32 TFLite
# ============================

model.save("efficientnetb3_cattle.keras")

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("efficientnetb3_cattle_fp32.tflite","wb") as f:
    f.write(tflite_model)

print("TFLite Exported!")


In [ ]:
# ============================
# Cell 12: Export labels
# ============================

labels_json = {i: name for i, name in enumerate(class_names)}

with open("labels.json","w") as f:
    json.dump(labels_json, f, indent=4)

with open("labels.txt","w") as f:
    for name in class_names:
        f.write(name+"\n")

print(labels_json)
